In [3]:
# 1 
# Importing all the required libraries 
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
import scipy.sparse as sparse

In [4]:
_stem_text_df = pd.read_csv('Stem_Text.csv')
# Remove the files without any Eglish text (feature vector)
_stem_text_df["English"] = _stem_text_df["Text"].str.contains('english version document avail sinc includ english special edit', regex=True)
#_stem_text_df['English'].value_counts()

In [5]:
_label_df = pd.read_csv("Eurovoc_Labels.csv")

In [6]:
_label_df['pivotcol'] = 1
_lbl_trnsps_df = pd.pivot_table(_label_df,values='pivotcol',index=['Filename'],columns=['Eurovoc_Label'],aggfunc=np.sum, fill_value=0)
_label_df = _label_df.groupby(['Filename'])['Eurovoc_Label'].apply(list).reset_index()

In [7]:
# merge the features and labels transposed 
df_for_stemming = _stem_text_df[_stem_text_df['English']==False]
_stem_text_df = df_for_stemming.drop(columns='English')
_array_df = pd.merge(_stem_text_df, _lbl_trnsps_df, on='Filename')
_array_df = _array_df.drop(columns='Filename')

In [8]:
Y_labels = pd.merge(_stem_text_df, _label_df, on='Filename')
Y_labels = Y_labels.drop(columns = ["Text","Filename"])

In [9]:
corr = pd.read_csv("Corr")
corr = corr.drop(columns = "Unnamed: 0")

In [10]:
#### for dropping the correlated labels 

col = corr.columns
cols_to_drop = []
k =0
num_classes = 0
for i in range(0,len(corr)):
  for j in range(0,len(corr)):
   # try: 
      if((float(corr[col[i]][j])>.9) and  (col[j] != i) and i<j):
        if(sum(_array_df[col[i]])) < sum(_array_df[col[j]]):
          cols_to_drop.append(col[i])
        else:
          cols_to_drop.append(col[j])
        
      #print(corr[i][j])
      #print("feture1",i,"feature 2",col[j])
        num_classes = num_classes +1

In [11]:
# gathered the columns above. Now drop
_array_df = _array_df.drop(columns = cols_to_drop)

In [12]:
### List drops

for j in cols_to_drop:
  for i in Y_labels["Eurovoc_Label"]:
  
    if(j in i):
      #print(i)
      #print("removing",j)
      i.remove(j)
      #print(i)
Y = Y_labels

In [13]:
import gc
gc.collect()

7

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(Y.Eurovoc_Label)
Y = multilabel_binarizer.transform(Y.Eurovoc_Label)

In [15]:
import scipy.sparse as sparse
X_tfidf = sparse.load_npz('sparse_matrix.npz')

In [16]:
from sklearn.model_selection import train_test_split
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(X_tfidf, Y, test_size=0.2, random_state=71)
# time after finishing
import gc
gc.collect()

25

In [17]:
print (Y.shape)
print (X_tfidf.shape)

(19399, 3926)
(19399, 10000)


In [26]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(penalty = "l2",loss='hinge', alpha=1e-4, random_state=42, max_iter=10, tol=None,class_weight="balanced")
clf = OneVsRestClassifier(sgd)
model = clf.fit(x_train_tfidf, y_train_tfidf)

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 15 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 33 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 36 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: Use

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 782 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 812 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 815 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 825 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 835 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:7

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2094 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2118 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2122 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2138 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2148 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3218 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3219 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3234 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3287 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3307 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass

In [27]:
#predicting
y_pred = clf.predict(x_test_tfidf)
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import jaccard_similarity_score
print("Hamming Loss Hammingloss",hamming_loss(y_test_tfidf,y_pred))
print("f1_score Micro Balanced ",f1_score(y_test_tfidf,y_pred,average='micro'))
print("f1_score Macro Balanced",f1_score(y_test_tfidf,y_pred,average='macro'))
print("f1_score WEighted Balanced ",f1_score(y_test_tfidf,y_pred,average='weighted'))
print("Jaccard score ",jaccard_similarity_score(y_test_tfidf,y_pred))


Hamming Loss Hammingloss 0.06986741837393848
f1_score Micro Balanced  0.03066618577132698


/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


f1_score Macro Balanced 0.09708200630477344
f1_score WEighted Balanced  0.36108096515523097


/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


Jaccard score  0.034785641429166604


In [28]:
import pickle
filename = 'model_SGDC_balanced_10iter_l2pen_hingels_1e4.sav'
pickle.dump(model, open(filename, 'wb'))

In [29]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(penalty = "l2",loss='hinge', alpha=1e-4, random_state=42, max_iter=5, tol=None,class_weight="balanced")
clf = OneVsRestClassifier(sgd)
model = clf.fit(x_train_tfidf, y_train_tfidf)

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 15 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 33 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 34 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 36 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: Use

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 782 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 812 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 815 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 825 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 835 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:7

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2094 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2118 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2122 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2138 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2148 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass

/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3218 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3219 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3234 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3287 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 3307 is present in all training examples.
  str(classes[c]))
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/multiclass

In [30]:
y_pred = clf.predict(x_test_tfidf)
from sklearn.metrics import f1_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import jaccard_similarity_score
print("Hamming Loss Hammingloss",hamming_loss(y_test_tfidf,y_pred))
print("f1_score Micro Balanced ",f1_score(y_test_tfidf,y_pred,average='micro'))
print("f1_score Macro Balanced",f1_score(y_test_tfidf,y_pred,average='macro'))
print("f1_score WEighted Balanced ",f1_score(y_test_tfidf,y_pred,average='weighted'))
print("Jaccard score ",jaccard_similarity_score(y_test_tfidf,y_pred))

Hamming Loss Hammingloss 0.09198372205387294
f1_score Micro Balanced  0.02424252553466635


/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


f1_score Macro Balanced 0.07081868311302265
f1_score WEighted Balanced  0.30624944078684546


/home/lhag/PythonProjects/ATML/ATML_virtual/lib/python3.6/site-packages/sklearn/metrics/classification.py:635: DeprecationWarning: jaccard_similarity_score has been deprecated and replaced with jaccard_score. It will be removed in version 0.23. This implementation has surprising behavior for binary and multiclass classification tasks.
  'and multiclass classification tasks.', DeprecationWarning)


Jaccard score  0.019903835087155826
